In [2]:
import numpy as np
import pandas as pd
import io

In [3]:
from google.colab import files
uploaded = files.upload()


Saving ufc_fight_data.csv to ufc_fight_data.csv
Saving ufc_fight_stat_data.csv to ufc_fight_stat_data.csv
Saving ufc_fighter_data.csv to ufc_fighter_data.csv
Saving ufc_event_data.csv to ufc_event_data.csv


In [4]:
print(uploaded.keys())
ufc_fight_data = pd.read_csv(io.BytesIO(uploaded['ufc_fight_data.csv']))
# ufc_fight_data
ufc_fight_stat_data = pd.read_csv(io.BytesIO(uploaded['ufc_fight_stat_data.csv']))
ufc_fighter_data = pd.read_csv(io.BytesIO(uploaded['ufc_fighter_data.csv']))
ufc_event_data = pd.read_csv(io.BytesIO(uploaded['ufc_event_data.csv']))

dict_keys(['ufc_fight_data.csv', 'ufc_fight_stat_data.csv', 'ufc_fighter_data.csv', 'ufc_event_data.csv'])
